In [10]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
data_covid_006 = pd.read_csv("/home/juan/proyectoTesis/corona_tested_individuals_ver_006.english.csv")
data_covid_0083 = pd.read_csv("/home/juan/proyectoTesis/corona_tested_individuals_ver_0083.english.csv")

/tmp/ipykernel_5422/1115030840.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data_covid_006 = pd.read_csv("/home/juan/proyectoTesis/corona_tested_individuals_ver_006.english.csv")
/tmp/ipykernel_5422/1115030840.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data_covid_0083 = pd.read_csv("/home/juan/proyectoTesis/corona_tested_individuals_ver_0083.english.csv")


In [3]:
print(data_covid_006.columns)
print(data_covid_006.shape)
print("*****************")
print(data_covid_0083.shape)
print(data_covid_0083.columns)

Index(['test_date', 'cough', 'fever', 'sore_throat', 'shortness_of_breath',
       'head_ache', 'corona_result', 'age_60_and_above', 'gender',
       'test_indication'],
      dtype='object')
(278848, 10)
*****************
(2742596, 10)
Index(['test_date', 'cough', 'fever', 'sore_throat', 'shortness_of_breath',
       'head_ache', 'corona_result', 'age_60_and_above', 'gender',
       'test_indication'],
      dtype='object')


In [4]:
data_covid_006.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278848 entries, 0 to 278847
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   test_date            278848 non-null  object 
 1   cough                278596 non-null  float64
 2   fever                278596 non-null  float64
 3   sore_throat          278847 non-null  float64
 4   shortness_of_breath  278847 non-null  float64
 5   head_ache            278847 non-null  float64
 6   corona_result        278848 non-null  object 
 7   age_60_and_above     151528 non-null  object 
 8   gender               259285 non-null  object 
 9   test_indication      278848 non-null  object 
dtypes: float64(5), object(5)
memory usage: 21.3+ MB


In [5]:
data_covid_006

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
0,2020-04-30,0.0,0.0,0.0,0.0,0.0,negative,NaN,female,Other
1,2020-04-30,1.0,0.0,0.0,0.0,0.0,negative,NaN,female,Other
2,2020-04-30,0.0,1.0,0.0,0.0,0.0,negative,NaN,male,Other
3,2020-04-30,1.0,0.0,0.0,0.0,0.0,negative,NaN,female,Other
4,2020-04-30,1.0,0.0,0.0,0.0,0.0,negative,NaN,male,Other
...,...,...,...,...,...,...,...,...,...,...
278843,2020-03-11,0.0,0.0,0.0,0.0,0.0,negative,NaN,NaN,Other
278844,2020-03-11,0.0,0.0,0.0,0.0,0.0,negative,NaN,NaN,Other
278845,2020-03-11,0.0,0.0,0.0,0.0,0.0,positive,NaN,NaN,Contact with confirmed
278846,2020-03-11,0.0,0.0,0.0,0.0,0.0,other,NaN,NaN,Other


In [6]:
datos_limpios = data_covid_006.copy()

datos_limpios = datos_limpios[datos_limpios['corona_result'].isin(['positive', 'negative'])]

datos_limpios = datos_limpios.dropna()

#FILTRO DE FECHAS
datos_limpios['test_date'] = pd.to_datetime(datos_limpios['test_date'])

fecha_inicio = '2020-03-22'
fecha_fin = '2020-04-07'
datos_limpios = datos_limpios[
    (datos_limpios['test_date'] >= fecha_inicio) &
    (datos_limpios['test_date'] <= fecha_fin)
]

#

print(f"Registros finales: {len(datos_limpios)}")
print("\nConteo de Clases (Desbalanceado):")
print(datos_limpios['corona_result'].value_counts())

Registros finales: 98989

Conteo de Clases (Desbalanceado):
corona_result
negative    90624
positive     8365
Name: count, dtype: int64


In [7]:
datos_limpios['test_indication'] = datos_limpios['test_indication'].apply(lambda x: 1 if x== 'Contact with confirmed' else 0)
datos_limpios['corona_result'] = datos_limpios['corona_result'].apply(lambda x: 1 if x== 'positive' else 0)
datos_limpios['age_60_and_above'] = datos_limpios['age_60_and_above'].apply(lambda x: 1 if x== 'Yes' else 0)
datos_limpios['gender'] = datos_limpios['gender'].apply(lambda x: 1 if x== 'male' else 0)

cols_a_entero = ['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache']

for col in cols_a_entero:
    datos_limpios[col] = datos_limpios[col].astype(int)

In [8]:
for col in datos_limpios.columns:
    print(f"{col}: Tiene {datos_limpios[col].unique()}")

test_date: Tiene <DatetimeArray>
['2020-04-07 00:00:00', '2020-04-06 00:00:00', '2020-04-05 00:00:00',
 '2020-04-04 00:00:00', '2020-04-03 00:00:00', '2020-04-02 00:00:00',
 '2020-04-01 00:00:00', '2020-03-31 00:00:00', '2020-03-30 00:00:00',
 '2020-03-29 00:00:00', '2020-03-28 00:00:00', '2020-03-27 00:00:00',
 '2020-03-26 00:00:00', '2020-03-25 00:00:00', '2020-03-24 00:00:00',
 '2020-03-23 00:00:00', '2020-03-22 00:00:00']
Length: 17, dtype: datetime64[ns]
cough: Tiene [0 1]
fever: Tiene [0 1]
sore_throat: Tiene [0 1]
shortness_of_breath: Tiene [0 1]
head_ache: Tiene [0 1]
corona_result: Tiene [0 1]
age_60_and_above: Tiene [0 1]
gender: Tiene [1 0]
test_indication: Tiene [0 1]


In [23]:
#SPLIT DE DATOS ANTES DE BALANCEAR

X = datos_limpios.drop(['corona_result', 'test_date'], axis=1)
y = datos_limpios['corona_result']


X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.3,
                                                    stratify = y,
                                                    random_state = 43)

print(f"Dimensiones originales X: {X_train.shape}")
print(f"Conteo original de clases: {y_train.value_counts().to_dict()}")

Dimensiones originales X: (69292, 8)
Conteo original de clases: {0: 63437, 1: 5855}


In [25]:
#Balanceo del Train
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)


In [ ]:
print(f"Nuevas originales X: {X_train_res.shape}")
print(f"Conteo original de clases: {y_train_res.value_counts().to_dict()}")

Dimensiones originales X: (126874, 8)
Conteo original de clases: {0: 63437, 1: 63437}


In [27]:
datos_train_balanceados = pd.concat([
    pd.DataFrame(X_train_res, columns=X.columns),
    pd.DataFrame(y_train_res, columns=['corona_result'])
], axis=1)

datos_test = pd.concat([
    pd.DataFrame(X_test, columns=X.columns),
    pd.DataFrame(y_test, columns=['corona_result'])
], axis=1)


#Guardar data set balanceado

datos_train_balanceados.to_csv('dataset2_covid_train_balanceado_final.csv', index=False)
datos_test.to_csv('dataset2_covid_test_final.csv')